In [34]:
import requests
import logging
logger = logging.getLogger(__name__)

from langchain_text_splitters import HTMLSectionSplitter

In [35]:
bp_uaa_server_ip = '10.186.34.99'
web_help_endpoint = 'uaa-ui/webhelp/WebHelp/'
menus_endpoint = 'Data/Tocs/BPUAA_User_Guide__Online_Output__Chunk0.js'

In [48]:
url = f'https://{bp_uaa_server_ip}/{web_help_endpoint}/{menus_endpoint}'
menus = requests.get(url, verify=False)
menus = menus.text.replace('define(', '').replace('});', '')
pages = {}
for n in menus.split('.htm'):
    if '/' in n:
        try:
            menu = n[n.find('/') + 1:] + '.htm'
            url = f'https://{bp_uaa_server_ip}/{web_help_endpoint}{menu}'
            #page = requests.get(url, verify=False).text
            pages[url] = ''
        except Exception as e:
            logger.error(f'Failed to process url {n} due to {str(e)}')
pages[f'https://{bp_uaa_server_ip}/{web_help_endpoint}/Home.htm'] = ''

/Users/bwubete/opt/anaconda3/envs/bpa_ai_sdk_new/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.186.34.99'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [49]:
pages

{'https://10.186.34.99/uaa-ui/webhelp/WebHelp/AI_Apps/AIApps.htm': '',
 'https://10.186.34.99/uaa-ui/webhelp/WebHelp/AI_Apps/Alarm_Prioritization/AlarmPrioritization.htm': '',
 'https://10.186.34.99/uaa-ui/webhelp/WebHelp/AI_Apps/Alarm_Prioritization/WorkingWithUseAlarmPrioritization.htm': '',
 'https://10.186.34.99/uaa-ui/webhelp/WebHelp/AI_Apps/Cross_Layer_Stitching/CrossLayerStitching.htm': '',
 'https://10.186.34.99/uaa-ui/webhelp/WebHelp/AI_Apps/Cross_Layer_Stitching/WorkingWithCrossLayerStitching.htm': '',
 'https://10.186.34.99/uaa-ui/webhelp/WebHelp/AI_Apps/Fiber_Degradation/FiberDegradation.htm': '',
 'https://10.186.34.99/uaa-ui/webhelp/WebHelp/AI_Apps/Fiber_Degradation/WorkingWithUseFiberDegradation.htm': '',
 'https://10.186.34.99/uaa-ui/webhelp/WebHelp/AI_Apps/Network_Health_Analytics/Network_Health_Analytics.htm': '',
 'https://10.186.34.99/uaa-ui/webhelp/WebHelp/AI_Apps/Silent_Fault_Detection/SilentFaultDetection.htm': '',
 'https://10.186.34.99/uaa-ui/webhelp/WebHelp/AI

In [37]:
# headers_to_split_on = [
#     ("h1", "Header 1"),
#     ("h2", "Header 2"),
#     ("h3", "Header 3"),
#     ("h4", "Header 4"),
#     ("table", "table"),
#     ('ul', 'unordered list'),
#     ('ol', 'ordered list')
# ]
# 
# html_splitter = HTMLSectionSplitter(headers_to_split_on=headers_to_split_on)
# splits = []
# for url in pages.keys():
#     html_header_splits = html_splitter.split_text(requests.get(url, verify=False).text)
#     splits.append(html_header_splits)

In [38]:
from langchain_community.document_loaders import AsyncHtmlLoader
loader = AsyncHtmlLoader(list(pages.keys()), requests_kwargs={'verify_ssl': False})
docs = loader.load()

Fetching pages: 100%|##########| 215/215 [00:18<00:00, 11.77it/s]


In [39]:
from langchain_community.document_transformers import MarkdownifyTransformer
# TODO: learn how to use images
md = MarkdownifyTransformer(strip=['img'])
converted_docs = md.transform_documents(docs)
print(converted_docs[0].page_content[:1000])

AI app manager

[Skip To Main Content](#)

Account
Settings

---

Logout

* placeholder

Account
Settings

---

Logout

Filter: 

* All Files

Submit Search

You are here: 

# [AI app manager A page where you can view, start, stop, schedule, update, or upload Blue Planet and user\-created AI apps.](#)

UAA provides a suite of BP\-curated AI Apps out\-of\-the\-box. In the following section, we provide detailed explanation of how to use each BP AI App. It is important to note that you also have the option to create and upload custom AI Apps into the AI Studio. For more information, see **AI Studio Administrator Guide**.

The Blue Planet App Bar UI now includes the AI Studio. To manage the AI Apps select AI Studio from the App Bar UI.

An App can be started and stopped by any user. Editing BP\-created AI apps is limited to the Blue Planet team. Any user can manage custom\-uploaded apps.

For more information, see *AI Studio Administrator Guide \> AI Apps Manager*
section.

AI Apps Manager

In [40]:
len(converted_docs)

215

In [41]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
import re
def remove_org_metadata(md_header_splits):
    curated_md_documents = []
    document_title = md_header_splits[0].page_content.replace('\n', '').replace('  ', ' ').split('Release')[0].strip()
    md_header_splits = md_header_splits[1:]
    for doc in md_header_splits:
        if not ("table of contents" in doc.metadata.get("Header 1", '').lower() or
                "publication history" in doc.metadata.get("Header 1", '').lower() or
                "contacting blue planet" in doc.metadata.get("Header 1", '').lower() or
                "legal notices" in doc.metadata.get("Header 1",'').lower()):
            doc.metadata["Document Title"] = document_title
            curated_md_documents.append(doc)
    return curated_md_documents

def md_splitter(pdf_content_as_Md):

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, strip_headers=False)
    md_chunks = markdown_splitter.split_text(pdf_content_as_Md.page_content)
    for chunk in md_chunks:
        chunk.metadata = {**pdf_content_as_Md.metadata, **chunk.metadata}
    return remove_org_metadata(md_chunks)


def remove_reference(text):
    return re.sub(r'!\[.*?\]\(.*?\)', '', text)

In [42]:
splits = []
for doc in converted_docs:
    splits.append(md_splitter(doc))

In [43]:
pdf_path = '/home/smohamme/RAG/SAM_RAG_CIENA/Ciena_pdfs'
Save_intermediate_Files = True
Create_VectorDB = True
persist_directory = None
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
use_gpu = True
model_name = embedding_model_name

model_kwargs = {"device": 'cpu'}

free_md_chunks = splits

In [44]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma

# merge list of lists splitted_md into a single list
merged_md = []
for md in free_md_chunks:
    merged_md.extend(md)


embedding = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs
)

summary_db = Chroma.from_documents(merged_md, embedding)

/Users/bwubete/opt/anaconda3/envs/bpa_ai_sdk_new/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [45]:
from langchain_core.documents import Document
from typing import List
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from llm_model import BpaOpenAILLM


c_rtrvr = summary_db.as_retriever()
llm = BpaOpenAILLM(model_name='test-model', model_id = '5', base_address='https://gptapi.ciena.com/', okta_domain='ciena.okta.com', okta_custom_scope='api', client_secret='uNU1c9Vl01WfkBdQDUivZr0O1pR3qxJpe5js3d1PJhwqGO63WEvqvOXgBux9dRyi', client_id='0oa1x6yhddqyoeoTY0h8')

xml_system = """You're a helpful AI assistant. Given a user question and some article snippets, \
answer the user question and provide citations. If none of the articles answer the question, just say you don't know.

Remember, you must return both an answer and citations. A citation consists of a VERBATIM quote that \
justifies the answer and the ID of the quote article. Return a citation for every quote across all articles \
that justify the answer. Use the following format for your final output:

<cited_answer>
    <answer></answer>
    <citations>
        <citation><source_id></source_id><quote></quote></citation>
        <citation><source_id></source_id><quote></quote></citation>
        ...
    </citations>
</cited_answer>

Here are the articles:{context}"""

def format_docs_xml(docs: List[Document]) -> str:
    formatted = []
    for i, doc in enumerate(docs):
        doc_str = f"""\
    <source id=\"{i}\">
        <title>{doc.metadata['Document Title']}</title>
        <article_snippet>{doc.page_content}</article_snippet>
    </source>"""
        formatted.append(doc_str)
    return "\n\n<sources>" + "\n".join(formatted) + "</sources>"

xml_prompt = ChatPromptTemplate.from_messages(
    [("system", xml_system), ("human", "{input}")]
)


rag_chain_from_docs = (
        RunnablePassthrough.assign(context=(lambda x: format_docs_xml(x["context"])))
        | xml_prompt
        | llm
)
retrieve_docs = (lambda x: x["input"]) | c_rtrvr

chain = RunnablePassthrough.assign(context=retrieve_docs).assign(
    answer=rag_chain_from_docs
)


In [52]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

system_prompt = (
    "Use the given context to answer the question. Only use information from the context. Return the answer in github markdown"
    "If you don't know the answer, say you don't know. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)



compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=c_rtrvr,
)
# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=compression_retriever,
#     return_source_documents=True
# )

question_answer_chain = create_stuff_documents_chain(llm, prompt)
qa_chain = create_retrieval_chain(summary_db.as_retriever(), question_answer_chain)

In [53]:
query = 'give me a table of topology settings'
# Perform RAG: Run RAG Chain
llm_answer = qa_chain.invoke({'input': query})

llm_answer

{'input': 'give me a table of topology settings',
 'context': [Document(metadata={'Document Title': 'Adding topology view [Skip To Main Content](#) AccountSettings --- Logout * placeholder AccountSettings --- Logout Filter: * All Files Submit Search You are here:', 'Header 2': 'Adding topology view', 'language': 'en-us', 'source': 'https://10.186.34.99/uaa-ui/webhelp/WebHelp/Topology/AddingTopology.htm', 'title': 'Adding topology view'}, page_content='## Adding topology view  \nThis section describes the procedure to add a subnetwork or node to topology view, which can be saved for future viewing.  \nFollow the given steps to add a new Topology view:  \n1. Open **Topology** from the Navigation Menu.\n2. Click **\\+** icon to create a new Topology.  \nTopology \\- Creating a new Topology view\n3. Select a Subnet / Node from Network Tree, perform a global search or apply query filter to create a topology view.\n4. In the **Name** field, enter a Name for the topology.  \nUser can use a co

In [47]:
from langchain_core.output_parsers import XMLOutputParser


def xml_parser(ai_response_with_citation):
    prs = XMLOutputParser()
    if  isinstance(ai_response_with_citation, dict):
        output = prs.parse(text=ai_response_with_citation['answer'].content)
    else:
        output = prs.parse(text=ai_response_with_citation)
    bot_response = output.get('cited_answer')[0]['answer']

    quote = output.get('cited_answer')[1].get('citations')[0].get('citation')[1].get('quote')
    reference_id = output.get('cited_answer')[1].get('citations')[0].get('citation')[0].get('source_id')

    return bot_response, quote, reference_id
xml_parser(llm_answer)

('Based on the provided articles, here is a table summarizing the topology settings:\n\n| Setting | Description |\n| --- | --- |\n| **Layer Filter** | Helps in assigning different layers to services, which reflect on the Topology page. |\n| **Network Tree Filter** | Allows filtering based on the network tree structure. |\n| **Global Search** | Enables searching across the entire network. |\n| **Query Filter** | Allows applying specific queries to filter the topology view. |\n| **Hop-Count Filter** | Filters based on the number of hops from a single node. |\n| **Logical/Physical View** | Option to switch between logical and physical views of the topology. |\n| **User Permissions** | Settings for user access permissions to the topology. |\n| **Cluster** | Option to enable or disable clusters of nodes based on geographical location. |\n| **Information Panel Tabs** | Includes Summary, Alarms, Metrics, Endpoints, and Interfaces tabs for detailed node information. |\n\n    ',
 'User can use 

In [105]:
def citation_generator(reference_id, ai_response_with_citation):
    try:
        reference_id = int(reference_id)
        cited_doc = ai_response_with_citation.get('context')[reference_id]
        citation = ''
        for key, value in cited_doc.metadata.items():
            citation += f' **{key}**: {value} - '
        return citation
    except:
        return None

In [106]:
bot_response, quote, ref_id = xml_parser(llm_answer)

In [107]:
citation = citation_generator(ref_id, llm_answer)

In [108]:
citation

' **Document Title**: AI app manager [Skip To Main Content](#) AccountSettings --- Logout * placeholder AccountSettings --- Logout Filter: * All Files Submit Search You are here: -  **Header 1**: [AI app manager A page where you can view, start, stop, schedule, update, or upload Blue Planet and user\\-created AI apps.](#) -  **language**: en-us -  **source**: https://10.186.34.99/uaa-ui/webhelp/WebHelp/AI_Apps/AIApps.htm -  **title**: AI app manager - '

In [109]:
bot_response

'The AI Apps currently supported by Blue Planet include:\n\n1. PM Monitoring App\n2. Cross Layer Stitching App\n\nThese apps can be managed through the AI Studio in the Blue Planet App Bar UI.'